<h1><strong>Análise de sentimentos em avaliações de e-commerce de roupas</strong></h1>


**Objetivo**:

Classificar as avaliações de e-commerce de roupas em positiva ou negativa.

*   Dataset: https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews

***Orientador:*** Thiago de Melo

***Orientando:*** Edson Brilhante

# Importando as bibliotecas

In [1]:
!pip install flair

     |████████████████████████████████| 322 kB 24.6 MB/s 
     |████████████████████████████████| 48 kB 4.5 MB/s 
     |████████████████████████████████| 1.2 MB 35.6 MB/s 
     |████████████████████████████████| 3.5 MB 54.0 MB/s 
     |████████████████████████████████| 981 kB 64.3 MB/s 
     |████████████████████████████████| 19.7 MB 1.2 MB/s 
     |████████████████████████████████| 53 kB 1.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 788 kB 43.1 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 596 kB 45.0 MB/s 
     |████████████████████████████████| 6.8 MB 55.6 MB/s 
     |████████████████████████████████| 895 kB 66.4 MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9692 sha256=6b344760f99d3f08a1c3ae3fdddfb4aa1c708b6545d90

In [2]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from flair.models import TextClassifier
from flair.data import Sentence
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Importando o *dataset*

In [3]:
dataset = pd.read_csv('/content/drive/MyDrive/Edson Henrique/dataset/Womens Clothing E-Commerce Reviews.csv')

# Visualização do *dataset*

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               23486 non-null  int64 
 1   Clothing ID              23486 non-null  int64 
 2   Age                      23486 non-null  int64 
 3   Title                    19676 non-null  object
 4   Review Text              22641 non-null  object
 5   Rating                   23486 non-null  int64 
 6   Recommended IND          23486 non-null  int64 
 7   Positive Feedback Count  23486 non-null  int64 
 8   Division Name            23472 non-null  object
 9   Department Name          23472 non-null  object
 10  Class Name               23472 non-null  object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


# Modelagem

## Seleciona as colunas '*Review Text*' e '*Rating*'

In [5]:
dataset = dataset[['Review Text', 'Rating']]
dataset = dataset.dropna()
dataset = dataset.reset_index(drop=True)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22641 entries, 0 to 22640
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review Text  22641 non-null  object
 1   Rating       22641 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 353.9+ KB


## Encoding

Seleciona o atributo *'Rating'* e o converte da seguinte forma:



*   1, 2 e 3: negativa
*   4 e 5: positiva

In [6]:
dataset['Rating'] = dataset['Rating'].apply(lambda x: 'pos' if x >= 4 else 'neg')
dataset['Rating'] = pd.Categorical(dataset['Rating'])

In [7]:
dataset

,Review Text,Rating
0,Absolutely wonderful - silky and sexy and comf...,pos
1,Love this dress! it's sooo pretty. i happene...,pos
2,I had such high hopes for this dress and reall...,neg
3,"I love, love, love this jumpsuit. it's fun, fl...",pos
4,This shirt is very flattering to all due to th...,pos
...,...,...
22636,I was very happy to snag this dress at such a ...,pos
22637,"It reminds me of maternity clothes. soft, stre...",neg
22638,"This fit well, but the top was very see throug...",neg
22639,I bought this dress for a wedding i have this ...,neg


## Treino

In [8]:
dataset['Flair'] = 0
dataset['Flair Rating'] = np.nan
dataset

,Review Text,Rating,Flair,Flair Rating
0,Absolutely wonderful - silky and sexy and comf...,pos,0,NaN
1,Love this dress! it's sooo pretty. i happene...,pos,0,NaN
2,I had such high hopes for this dress and reall...,neg,0,NaN
3,"I love, love, love this jumpsuit. it's fun, fl...",pos,0,NaN
4,This shirt is very flattering to all due to th...,pos,0,NaN
...,...,...,...,...
22636,I was very happy to snag this dress at such a ...,pos,0,NaN
22637,"It reminds me of maternity clothes. soft, stre...",neg,0,NaN
22638,"This fit well, but the top was very see throug...",neg,0,NaN
22639,I bought this dress for a wedding i have this ...,neg,0,NaN


In [9]:
classifier = TextClassifier.load('sentiment-fast')

2022-02-14 14:20:33,705 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-fasttext-rnn/sentiment-en-mix-ft-rnn_v8.pt not found in cache, downloading to /tmp/tmpqg254k23


100%|██████████| 1241977025/1241977025 [01:46<00:00, 11663365.66B/s]

2022-02-14 14:22:20,555 copying /tmp/tmpqg254k23 to cache at /root/.flair/models/sentiment-en-mix-ft-rnn_v8.pt


2022-02-14 14:22:25,431 removing temp file /tmp/tmpqg254k23
2022-02-14 14:22:25,661 loading file /root/.flair/models/sentiment-en-mix-ft-rnn_v8.pt


In [10]:
def flair_sentiment(texts, classifier):
  sentences = [Sentence(text) for text in texts]
  classifier.predict(sentences, mini_batch_size=32)
  return [
          (sent.labels[0].score, sent.labels[0].value)
          for sent in sentences
          ]

sentiments = flair_sentiment(dataset['Review Text'].values, classifier)

dataset['Flair'] = [sent[0] for sent in sentiments]
dataset['Flair Rating'] = [sent[1] for sent in sentiments]

In [11]:
dataset['Flair Rating'] = pd.Categorical(dataset['Flair Rating'])
dataset['Flair Rating'] = dataset['Flair Rating'].cat.rename_categories({'NEGATIVE': 'neg', 'POSITIVE': 'pos'})

In [12]:
y_true = dataset['Rating'].values
y_pred = dataset['Flair Rating'].values

In [13]:
print('Matriz de confusão')
print(confusion_matrix(y_true, y_pred))

print('\nMétricas')
print(classification_report(y_true, y_pred, digits=3))

print('\nScore')
print(accuracy_score(y_true, y_pred))

Matriz de confusão
[[ 4557   636]
 [ 2406 15042]]

Métricas
              precision    recall  f1-score   support

         neg      0.654     0.878     0.750      5193
         pos      0.959     0.862     0.908     17448

    accuracy                          0.866     22641
   macro avg      0.807     0.870     0.829     22641
weighted avg      0.889     0.866     0.872     22641


Score
0.8656419769444813


In [14]:
dataset[dataset['Flair Rating'] == 'pos']['Flair Rating'].count()

15678

In [15]:
dataset[dataset['Flair Rating'] == 'neg']['Flair Rating'].count()

6963